In [25]:
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [26]:
df = gpd.read_file('/app/notebooks/rapprochements/Grenoble/202406_BATIMfiltre-RNB_EPSG2154.gpkg')

print("nombre de bâtiments : ", len(df))

nombre de bâtiments :  15299


In [27]:
# print current crs
print("current crs : ", df.crs)

# convert from 3945 to 4326
df = df.to_crs(epsg=4326)

print(df.head())

current crs :  EPSG:3945
   BATIM_ID                                           geometry
0      88.0  MULTIPOLYGON (((5.69643 45.20544, 5.69654 45.2...
1      89.0  MULTIPOLYGON (((5.69717 45.20487, 5.69729 45.2...
2      90.0  MULTIPOLYGON (((5.69769 45.20447, 5.69782 45.2...
3      91.0  MULTIPOLYGON (((5.69594 45.20896, 5.69596 45.2...
4      92.0  MULTIPOLYGON (((5.69136 45.20406, 5.69132 45.2...


In [28]:
from batid.models import Building
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection

def intersects_with(geom):
    sql = f"SELECT rnb_id FROM batid_building WHERE ST_Intersects(shape, ST_GeomFromText('{geom.wkt}', 4326))"
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rnb_ids = cursor.fetchall()
    
    return rnb_ids

df['grenoble_rnb_intersection'] = df['geometry'].apply(intersects_with)
print(df.head())

   BATIM_ID                                           geometry  \
0      88.0  MULTIPOLYGON (((5.69643 45.20544, 5.69654 45.2...   
1      89.0  MULTIPOLYGON (((5.69717 45.20487, 5.69729 45.2...   
2      90.0  MULTIPOLYGON (((5.69769 45.20447, 5.69782 45.2...   
3      91.0  MULTIPOLYGON (((5.69594 45.20896, 5.69596 45.2...   
4      92.0  MULTIPOLYGON (((5.69136 45.20406, 5.69132 45.2...   

  grenoble_rnb_intersection  
0         [(Z3PDD3ZPPH7F,)]  
1         [(RNMWMCG8884F,)]  
2         [(SY7Y7H2TE7EQ,)]  
3         [(2AKTFNSC46MZ,)]  
4         [(FV92328RPJJJ,)]  


In [29]:
# count the number of lines without intersection
print("nombre de lignes sans intersection : ", len(df[df['grenoble_rnb_intersection'].apply(len) == 0]))

nombre de lignes sans intersection :  583


In [30]:
def link_1_1(geom):
    sql = f"""SELECT rnb_id FROM batid_building WHERE ST_Intersects(shape, ST_GeomFromText('{geom.wkt}', 4326))
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(ST_GeomFromText('{geom.wkt}', 4326)) > 0.8
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(shape) > 0.8    
    """
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rnb_ids = cursor.fetchall()

        return rnb_ids

def grenoble_inclu(geom):
    sql = f"""SELECT rnb_id FROM batid_building WHERE ST_AREA(shape) > 0 and ST_Intersects(shape, ST_GeomFromText('{geom.wkt}', 4326))
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(ST_GeomFromText('{geom.wkt}', 4326)) > 0.8
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(shape) <= 0.8    
    """
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rnb_ids = cursor.fetchall()

        return rnb_ids

def rnb_inclu(geom):
    sql = f"""SELECT rnb_id FROM batid_building WHERE ST_AREA(shape) > 0 and ST_Intersects(shape, ST_GeomFromText('{geom.wkt}', 4326))
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(ST_GeomFromText('{geom.wkt}', 4326)) <= 0.8
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(shape) > 0.8    
    """
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rnb_ids = cursor.fetchall()

        return rnb_ids

def small_intersection(geom):
    sql = f"""SELECT rnb_id FROM batid_building WHERE ST_AREA(shape) > 0 and ST_Intersects(shape, ST_GeomFromText('{geom.wkt}', 4326))
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(ST_GeomFromText('{geom.wkt}', 4326)) <= 0.8
    and ST_AREA(ST_Intersection(shape, ST_GeomFromText('{geom.wkt}', 4326))) / ST_AREA(shape) <= 0.8    
    """
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rnb_ids = cursor.fetchall()

        return rnb_ids

df['match_1_1'] = df['geometry'].apply(link_1_1)
df['grenoble_inclu_dans_RNB'] = df['geometry'].apply(grenoble_inclu)
df['RNB_inclu_dans_grenoble'] = df['geometry'].apply(rnb_inclu)
df['no_intersection'] = df['grenoble_rnb_intersection'].apply(len) == 0
df['small_intersection'] = df['geometry'].apply(small_intersection)

In [31]:
# count the number of 1_1 matches
print("nombre de 1_1 : ", len(df[df['match_1_1'].apply(lambda x: len(x) > 0)]))

nombre de 1_1 :  9969


In [34]:
df.head()

,BATIM_ID,geometry,grenoble_rnb_intersection,match_1_1,grenoble_inclu_dans_RNB,RNB_inclu_dans_grenoble,no_intersection,small_intersection
0,88.0,"MULTIPOLYGON (((5.69643 45.20544, 5.69654 45.2...","[(Z3PDD3ZPPH7F,)]","[(Z3PDD3ZPPH7F,)]",[],[],False,[]
1,89.0,"MULTIPOLYGON (((5.69717 45.20487, 5.69729 45.2...","[(RNMWMCG8884F,)]","[(RNMWMCG8884F,)]",[],[],False,[]
2,90.0,"MULTIPOLYGON (((5.69769 45.20447, 5.69782 45.2...","[(SY7Y7H2TE7EQ,)]","[(SY7Y7H2TE7EQ,)]",[],[],False,[]
3,91.0,"MULTIPOLYGON (((5.69594 45.20896, 5.69596 45.2...","[(2AKTFNSC46MZ,)]",[],[],[],False,"[(2AKTFNSC46MZ,)]"
4,92.0,"MULTIPOLYGON (((5.69136 45.20406, 5.69132 45.2...","[(FV92328RPJJJ,)]","[(FV92328RPJJJ,)]",[],[],False,[]


In [35]:
print(df.head())

   BATIM_ID                                           geometry  \
0      88.0  MULTIPOLYGON (((5.69643 45.20544, 5.69654 45.2...   
1      89.0  MULTIPOLYGON (((5.69717 45.20487, 5.69729 45.2...   
2      90.0  MULTIPOLYGON (((5.69769 45.20447, 5.69782 45.2...   
3      91.0  MULTIPOLYGON (((5.69594 45.20896, 5.69596 45.2...   
4      92.0  MULTIPOLYGON (((5.69136 45.20406, 5.69132 45.2...   

  grenoble_rnb_intersection          match_1_1 grenoble_inclu_dans_RNB  \
0         [(Z3PDD3ZPPH7F,)]  [(Z3PDD3ZPPH7F,)]                      []   
1         [(RNMWMCG8884F,)]  [(RNMWMCG8884F,)]                      []   
2         [(SY7Y7H2TE7EQ,)]  [(SY7Y7H2TE7EQ,)]                      []   
3         [(2AKTFNSC46MZ,)]                 []                      []   
4         [(FV92328RPJJJ,)]  [(FV92328RPJJJ,)]                      []   

  RNB_inclu_dans_grenoble  no_intersection small_intersection  
0                      []            False                 []  
1                      []     

In [36]:
# remove parenthesis
df['grenoble_rnb_intersection'] = df['grenoble_rnb_intersection'].apply(lambda x: [y[0] for y in x])
df['match_1_1'] = df['match_1_1'].apply(lambda x: [y[0] for y in x])
df['grenoble_inclu_dans_RNB'] = df['grenoble_inclu_dans_RNB'].apply(lambda x: [y[0] for y in x])
df['RNB_inclu_dans_grenoble'] = df['RNB_inclu_dans_grenoble'].apply(lambda x: [y[0] for y in x])
df['small_intersection'] = df['small_intersection'].apply(lambda x: [y[0] for y in x])

df.head()

,BATIM_ID,geometry,grenoble_rnb_intersection,match_1_1,grenoble_inclu_dans_RNB,RNB_inclu_dans_grenoble,no_intersection,small_intersection
0,88.0,"MULTIPOLYGON (((5.69643 45.20544, 5.69654 45.2...",[Z3PDD3ZPPH7F],[Z3PDD3ZPPH7F],[],[],False,[]
1,89.0,"MULTIPOLYGON (((5.69717 45.20487, 5.69729 45.2...",[RNMWMCG8884F],[RNMWMCG8884F],[],[],False,[]
2,90.0,"MULTIPOLYGON (((5.69769 45.20447, 5.69782 45.2...",[SY7Y7H2TE7EQ],[SY7Y7H2TE7EQ],[],[],False,[]
3,91.0,"MULTIPOLYGON (((5.69594 45.20896, 5.69596 45.2...",[2AKTFNSC46MZ],[],[],[],False,[2AKTFNSC46MZ]
4,92.0,"MULTIPOLYGON (((5.69136 45.20406, 5.69132 45.2...",[FV92328RPJJJ],[FV92328RPJJJ],[],[],False,[]


In [37]:
df[['BATIM_ID', 'grenoble_rnb_intersection', 'match_1_1', 'grenoble_inclu_dans_RNB', 'RNB_inclu_dans_grenoble', 'no_intersection', 'small_intersection']].to_csv('/app/notebooks/rapprochements/Grenoble/resultat_rapprochement_rnb_grenoble.csv', index=False)


In [24]:
# number of True by column
print("nombre de True par colonne : ")
print(df[['match_1_1', 'grenoble_inclu_dans_RNB', 'RNB_inclu_dans_grenoble', 'no_intersection', 'small_intersection']].apply(pd.Series.value_counts))

nombre de True par colonne : 
       match_1_1  grenoble_inclu_dans_RNB  RNB_inclu_dans_grenoble  \
False       5330                    14413                    14434   
True        9969                      886                      865   

       no_intersection  small_intersection  
False            14716                5065  
True               583               10234  
